In [1]:
#Reset kernel when start this notebook
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder.master('local[*]').getOrCreate()

sc = spark.sparkContext

spark

In [62]:
df = spark.read.format("csv").option("inferSchema", "true").option("timestampFormat","yyyy-MM-dd HH:mm:ss").option("header", "true").option("mode", "DROPMALFORMED").load("content/tripdata_2017-01.csv")

In [4]:
df.rdd.take(1)

[Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2017, 1, 9, 11, 13, 28), tpep_dropoff_datetime=datetime.datetime(2017, 1, 9, 11, 25, 45), passenger_count=1, trip_distance=3.3, RatecodeID=1, store_and_fwd_flag='N', PULocationID=263, DOLocationID=161, payment_type=1, fare_amount=12.5, extra=0.0, mta_tax=0.5, tip_amount=2.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=15.3)]

In [58]:
df.rdd.map(lambda x: x[7]).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1], False).take(10)

[(79, 40918),
 (230, 34460),
 (48, 33407),
 (186, 32940),
 (234, 30479),
 (161, 29329),
 (170, 28218),
 (237, 28193),
 (142, 26926),
 (236, 26652)]

In [59]:
df.rdd.map(lambda x: x[1].hour).take(1)

[11]

In [104]:
filtered = df.rdd.filter(lambda x: x[1] is not None and x[2] is not None and x[4] is not None).filter(lambda x: (x[2] - x[1]).seconds > 0)

In [124]:
mapped = filtered.map(lambda x: (x[1].hour, (round(x[4]/(((x[2] - x[1]).seconds)/3600), 2), 1)))

In [125]:
mapped.take(10)

[(11, (16.12, 1)),
 (11, (15.14, 1)),
 (11, (17.6, 1)),
 (11, (12.26, 1)),
 (0, (7.89, 1)),
 (0, (11.82, 1)),
 (0, (6.94, 1)),
 (0, (6.38, 1)),
 (0, (21.19, 1)),
 (0, (32.75, 1))]

In [127]:
result = mapped.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [131]:
result.map(lambda x: (x[0], x[1][0] / x[1][1] )).sortBy(lambda x: x[0], True).take(24)

[(0, 15.77269862499405),
 (1, 14.893820369745224),
 (2, 15.756142360064649),
 (3, 17.52552260175904),
 (4, 20.431065719360596),
 (5, 22.738685249526185),
 (6, 19.460559736050012),
 (7, 14.693573043512679),
 (8, 14.160713046325105),
 (9, 13.465949734596334),
 (10, 14.408588508538111),
 (11, 13.260461307752731),
 (12, 13.200534720701002),
 (13, 12.857011267107083),
 (14, 14.23640506498278),
 (15, 12.733836160782529),
 (16, 16.185624409110382),
 (17, 13.222246619547837),
 (18, 13.14339796215427),
 (19, 15.387991450938916),
 (20, 14.675617669704746),
 (21, 15.237855983886298),
 (22, 17.185417126697224),
 (23, 19.342074761502978)]